In [1]:
#given an input catalogue, generates the list of reference targets and appends
#to a new file appended with 'reference_targets' """
import numpy as np
from astropy.io import fits, ascii
import astropy.units as u
from astropy.coordinates import SkyCoord
from scipy.spatial import cKDTree
import os
import tqdm

In [2]:
# LOAD THE DATA
path = 'phatter-topright.fits'
tb = fits.open(path)
fullcat = tb[1].data 


FileNotFoundError: [Errno 2] No such file or directory: 'phatter-topright.fits'

In [3]:
refmag = 'F160W_VEGA'
brightest_in_3p2 = []
deltamag = []
data = np.dstack([fullcat['RA'], fullcat['DEC']])[0]
brighter = ( (fullcat[refmag] < 20.0) & (fullcat[refmag] > 19.3) )
skycoords = SkyCoord(ra=fullcat['RA'][brighter]*u.deg, dec=fullcat['DEC'][brighter]*u.deg)


NameError: name 'fullcat' is not defined

In [26]:
for indx, i in tqdm.tqdm(enumerate(range(len(skycoords)))):
    #make a mask to remove the star being checked
    checker = np.ones(len(skycoords), dtype=bool)
    checker[i] = False
    #make the list of stars to check (removing the target)
    checkcoords = skycoords[checker]
    seps = skycoords[i].separation(checkcoords)
    #get all stars in the list within 3.2 arcsec
    nearby = seps < 3.2*u.arcsec
    within2 = seps < 2*u.arcsec
    #check if the target object is the brightest
    magnitudes = fullcat[refmag][brighter][checker][nearby].data
    magnitudes2 = fullcat[refmag][brighter][checker][within2].data
    #if brightest in 3.2 arcsec, add to list
    if (magnitudes > fullcat[refmag][brighter][i]).all():
        brightest_in_3p2.append(indx)
        if len(magnitudes2) > 0:
            #compute magnitude difference between reference star and (next) brightest in 3.2 (for testing)
            deltamags = magnitudes2 - fullcat[refmag][brighter][i]
            deltamag.append(np.min(deltamags))
        else:
            deltamag.append(np.inf)

61it [00:00, 888.32it/s]


In [27]:
#make a list of all stars with no neighbours brighter than reference mag = 23.3 in 0.3 arcsec:
mask = fullcat[refmag] < 24.
data = np.dstack([fullcat['RA'], fullcat['DEC']])[0]
tree = cKDTree(data)
bright_in_0p3 = tree.query_ball_point(data, 0.3/3600)
print(bright_in_0p3)
close_neighbour = np.ones(len(bright_in_0p3), dtype=bool)
for i in range(len(bright_in_0p3)):
    if len(bright_in_0p3[i]) == 1:
        close_neighbour[i] = 0
brightest_in_3p2_mask = np.zeros(len(fullcat), dtype=bool)
brightest_in_3p2_mask[brightest_in_3p2] = 1
#finally, make the list of magnitude differences
all_deltamag = np.ones(len(fullcat))*np.nan
all_deltamag[brightest_in_3p2_mask] = deltamag
refmask = brightest_in_3p2_mask & ~close_neighbour & (fullcat[refmag] < 22.) & (fullcat[refmag] > 19.5) & (all_deltamag > 1.5)
reference = np.zeros(len(fullcat), dtype=bool)
reference[refmask] = 1
rec = np.recarray(len(fullcat), dtype=[('RA', float),('DEC', float),('ref_mag', float), ('Reference', int)])
rec['RA'] = fullcat['RA']
rec['DEC'] = fullcat['DEC']
rec['Reference'] = reference
# ascii.write(rec, '/Users/dhortadarrington/Desktop/M33_reference_targets.dat',  overwrite=True, format='csv')



[list([0]) list([1]) list([2]) list([3]) list([4]) list([5]) list([6])
 list([7]) list([8]) list([9]) list([10]) list([11]) list([12]) list([13])
 list([14]) list([15]) list([16]) list([17]) list([18]) list([19])
 list([20]) list([21]) list([22]) list([23]) list([24]) list([25])
 list([26]) list([27]) list([28]) list([29]) list([30]) list([31])
 list([32]) list([33]) list([34]) list([35]) list([36]) list([37])
 list([38]) list([39]) list([40]) list([41]) list([42]) list([43])
 list([44]) list([45]) list([46]) list([47]) list([48]) list([49])
 list([50]) list([51]) list([52]) list([53]) list([54]) list([55])
 list([56]) list([57]) list([58]) list([59]) list([60]) list([61])
 list([62]) list([63]) list([64]) list([65]) list([66]) list([67])
 list([68]) list([69]) list([70]) list([71]) list([72]) list([73])
 list([74]) list([75]) list([76]) list([77]) list([78]) list([79])
 list([80]) list([81]) list([82]) list([83]) list([84]) list([85])
 list([86]) list([87]) list([88]) list([89]) list(